In [1]:
!ls

README.md		    miniconda.sh  quick_start_pytorch.ipynb   wandb
eliciting-latent-sentiment  miniconda3	  quick_start_pytorch_images


In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [6]:
#!pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
!pip install fancy_einsum==0.0.3
!pip install transformer_lens
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git
!pip install circuitsvis
# !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# %pip install git+https://github.com/neelnanda-io/PySvelte.git
# %pip install typeguard==2.13.3

  Cloning https://github.com/neelnanda-io/neel-plotly.git to /tmp/pip-req-build-3aa7cepo
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly.git /tmp/pip-req-build-3aa7cepo
  Resolved https://github.com/neelnanda-io/neel-plotly.git to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0


In [7]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import einops
from functools import partial
import torch
import datasets
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_dataset, tokenize_and_concatenate, get_act_name, test_prompt
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
from circuitsvis.activations import text_neuron_activations
from utils.store import load_array, save_html, save_array, is_file, get_model_name, clean_label, save_text

In [9]:
def find_positions(tensor, token_ids=[11, 13]):
    positions = []
    for batch_item in tensor:
        token_positions = {token_id: [] for token_id in token_ids}
        for position, token in enumerate(batch_item):
            if token.item() in token_ids:
                token_positions[token.item()].append(position)
        positions.append([token_positions[token_id] for token_id in token_ids])
    return positions

In [10]:
def zero_attention_pos_hook(
    pattern: Float[Tensor, "batch head seq_Q seq_K"], hook: HookPoint,
    pos_by_batch: List[List[int]], layer: int = 0, head_idx: int = 0,
) -> Float[Tensor, "batch head seq_Q seq_K"]:
    """Zero-ablates an attention pattern tensor at a particular position"""
    assert 'pattern' in hook.name

    batch_size = pattern.shape[0]
    assert len(pos_by_batch) == batch_size

    for i in range(batch_size):
        for p in pos_by_batch[i]:
            pattern[i, head_idx, p, p] = 0
            
    return pattern



In [11]:
def names_filter(name: str):
    """Filter for the names of the activations we want to keep to study the resid stream."""
    return name.endswith('resid_post') or name == get_act_name('resid_pre', 0)

def get_layerwise_token_mean_activations(model: HookedTransformer, data_loader: DataLoader, token_id: int = 13) -> Float[Tensor, "layer d_model"]:
    """Get the mean value of a token across layers"""
    num_layers = model.cfg.n_layers
    d_model = model.cfg.d_model
    
    activation_sums = torch.stack([torch.zeros(d_model) for _ in range(num_layers)]).to(device)
    comma_counts = [0] * num_layers

    print(activation_sums.shape)

    token_mean_values = torch.zeros((num_layers, d_model))
    for _, batch_value in tqdm(enumerate(data_loader), total=100):
        
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        _, cache = model.run_with_cache(
            batch_tokens, 
            names_filter=names_filter
        )

        
        for i in range(batch_tokens.shape[0]):
            for p in punct_pos[i][0]:
                for layer in range(num_layers):
                    activation_sums[layer] += cache[f"blocks.{layer}.hook_resid_post"][i, p, :]
                    comma_counts[layer] += 1

    for layer in range(num_layers):
        token_mean_values[layer] = activation_sums[layer] / comma_counts[layer]

    return token_mean_values

In [12]:
def compute_zeroed_attn_modified_loss(model: HookedTransformer, data_loader: DataLoader) -> float:
    total_loss = 0
    loss_list = []
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=[13])

        # get the loss for each token in the batch
        initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        
        # add hooks for the activations of the 11 and 13 tokens
        for layer, head in heads_to_ablate:
            ablate_punct = partial(zero_attention_pos_hook, pos_by_batch=punct_pos, layer=layer, head_idx=head)
            model.blocks[layer].attn.hook_pattern.add_hook(ablate_punct)

        # get the loss for each token when run with hooks
        hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

        # compute the percent difference between the two losses
        loss_diff = (hooked_loss - initial_loss) / initial_loss

        loss_list.append(loss_diff)

    model.reset_hooks()
    return loss_list, batch_tokens

In [13]:
from utils.ablation import ablate_resid_with_precalc_mean

def compute_mean_ablation_modified_loss(model: HookedTransformer, data_loader: DataLoader, cached_means, target_token_ids) -> float:
    total_loss = 0
    loss_diff_list = []
    orig_loss_list = []
    for _, batch_value in tqdm(enumerate(data_loader), total=len(data_loader)):
        if isinstance(batch_value['tokens'], list):
            batch_tokens = torch.stack(batch_value['tokens']).to(device)
        else:
            batch_tokens = batch_value['tokens'].to(device)

        # get positions of all 11 and 13 token ids in batch
        punct_pos = find_positions(batch_tokens, token_ids=target_token_ids)

        # get the loss for each token in the batch
        initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
        orig_loss_list.append(initial_loss)
        
        # add hooks for the activations of the 11 and 13 tokens
        for layer, head in heads_to_ablate:
            mean_ablate_comma = partial(ablate_resid_with_precalc_mean, cached_means=cached_means, pos_by_batch=punct_pos, layer=layer)
            model.blocks[layer].hook_resid_post.add_hook(mean_ablate_comma)

        # get the loss for each token when run with hooks
        hooked_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

        # compute the difference between the two losses
        loss_diff = hooked_loss - initial_loss
        

        print(punct_pos)
        # set all positions right after punct_pos to zero
        for i in range(batch_tokens.shape[0]):
            for p in punct_pos[i][0]:
                print(f"zeroing {i}, {p}")
                loss_diff[i, p] = 0

        loss_diff_list.append(loss_diff)

    model.reset_hooks()
    return loss_diff_list, orig_loss_list

In [14]:
def plot_neuroscope(
    tokens: Int[Tensor, "batch pos"], centred: bool = False, activations: Float[Tensor, "pos layer 1"] = None,
    verbose=False,
):
    
    str_tokens = model.to_str_tokens(tokens, prepend_bos=False)

    if verbose:
        print(f"Tokens shape: {tokens.shape}")
  
    if centred:
        if verbose:
            print("Centering activations")
        layer_means = einops.reduce(activations, "pos layer 1 -> 1 layer 1", reduction="mean")
        layer_means = einops.repeat(layer_means, "1 layer 1 -> pos layer 1", pos=activations.shape[0])
        activations -= layer_means
    elif verbose:
        print("Activations already centered")
    assert (
        activations.ndim == 3
    ), f"activations must be of shape [tokens x layers x neurons], found {activations.shape}"
    assert len(str_tokens) == activations.shape[0], (
        f"tokens and activations must have the same length, found tokens={len(str_tokens)} and acts={activations.shape[0]}, "
        f"tokens={str_tokens}, "
        f"activations={activations.shape}"

    )
    return text_neuron_activations(
        tokens=str_tokens, 
        activations=activations,
        first_dimension_name="Layer (resid_pre)",
        second_dimension_name="Model",
        second_dimension_labels=["pythia-2.8b"],
    )

## Comma Ablation on Natural Text

In [26]:
torch.set_grad_enabled(False)
device = "cuda"
MODEL_NAME = "EleutherAI/pythia-2.8b"
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)
model.name = MODEL_NAME

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


In [17]:
from datasets import load_from_disk
sst_data = load_from_disk("sst2")
sst_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8104
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2125
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1043
    })
})

In [18]:
from typing import List
from datasets import Dataset, Features, Sequence, Value
from transformers import AutoTokenizer
import torch

def tokenize_and_concatenate2(dataset, tokenizer, max_length=1024, column_name='text', add_bos_token=True):
    token_buffer = []
    final_batches = []
    
    for batch in dataset:
        text = batch[column_name]
        if add_bos_token:
            text = tokenizer.bos_token + text
        tokenized_text = tokenizer(text, add_special_tokens=False)['input_ids']
        eos_token_id = tokenizer.eos_token_id
        tokenized_text.append(eos_token_id)
        token_buffer.extend(tokenized_text)
        
        while len(token_buffer) >= max_length:
            final_batch = token_buffer[:max_length]
            token_buffer = token_buffer[max_length:]
            final_batches.append(final_batch)
    
    # Handle any remaining tokens
    if len(token_buffer) > 0:
        final_batches.append(token_buffer)
    
    # Convert list of batches to tensors
    final_batches = [torch.tensor(batch) for batch in final_batches]
    
    # Create a new dataset with specified features
    features = Features({"tokens": Sequence(Value("int64"))})
    final_dataset = Dataset.from_dict({"tokens": final_batches}, features=features)

    final_dataset.set_format(type="torch", columns=["tokens"])
    
    return final_dataset

# # Example usage
# tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Make sure the tokenizer has bos_token_id and eos_token_id
# text_dataset = Dataset.from_dict({"text": ["This is a sample text.", "Another sample text."]})  # Example dataset
# tokenized_dataset = tokenize_and_concatenate2(text_dataset, tokenizer, max_length=1024)



In [57]:
sst_data['train'][0]['text']

"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."

In [34]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer


# Define the batch size
BATCH_SIZE = 5

# Load a tokenizer (you'll need to specify the appropriate model)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-2.8b")
# set padding token
tokenizer.pad_token = tokenizer.eos_token

#dataset = text_dataset.map(lambda x: tokenize_and_concatenate(x, tokenizer))

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024)

dataset = sst_data['train'].map(tokenize_function, batched=True)

# Create a PyTorch DataLoader
train_data_loader = DataLoader(
    dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True
)

# Now, train_data_loader is ready to be used for training or analysis


Map:   0%|          | 0/8104 [00:00<?, ? examples/s]

In [35]:
dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8104
})

In [36]:
# rename input_ids to tokens
dataset = dataset.rename_column("input_ids", "tokens")

In [37]:
dataset = dataset.select(range(193))

In [23]:
# comma_mean_values = get_layerwise_token_mean_activations(model, data_loader, token_id=13)
# save_array(comma_mean_values, 'comma_mean_values.npy', model)

In [24]:
period_mean_values = get_layerwise_token_mean_activations(model, train_data_loader, token_id=15)
save_array(period_mean_values, 'period_mean_values.npy', model)

NameError: name 'model' is not defined

In [38]:
# load the files
comma_mean_values = torch.from_numpy(load_array('comma_mean_values.npy', model)).to(device)
period_mean_values = torch.from_numpy(load_array('period_mean_values.npy', model)).to(device)

In [39]:
import random

# Generate random indices
n = 20  # Size of the random subset
total_size = len(dataset)
random_indices = random.sample(range(total_size), n)

# Get a random subset of the dataset of a given size
subset_dataset = dataset.select(random_indices)

# Create a new dataloader from the subset, converting the data to tensors
subset_data_loader = DataLoader(
    subset_dataset, batch_size=5, shuffle=False, drop_last=True
)
len(subset_data_loader)


4

In [40]:
heads_to_ablate = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]
loss_change_by_token, orig_loss = compute_mean_ablation_modified_loss(model, subset_data_loader, comma_mean_values, target_token_ids=[13])

  0%|          | 0/4 [00:00<?, ?it/s]

[[[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]

In [51]:
for i in range(len(subset_dataset)):
    print(model.to_string(subset_dataset[i]['tokens']))
    print(subset_dataset[i]['tokens'])

A welcome relief from baseball movies that try too hard to be mythic , this one is a sweet and modest and ultimately winning story .<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [55]:
for i in range(1157, 1158):
    print(f"Token {i}: {model.to_str_tokens(torch.tensor([i]))}")

Token 1157: [' ,']


In [42]:
for i in range(len(loss_change_by_token)):
    # add one column of zeros to the loss change tensor
    loss_change_by_token[i] = torch.cat([torch.zeros(loss_change_by_token[i].shape[0], 1).to(device), loss_change_by_token[i]], dim=1)

loss_change_by_token = torch.stack(loss_change_by_token).cpu()


In [49]:
loss_change_by_token.shape

torch.Size([100, 1024, 1])

In [50]:
loss_change_by_token_by_row = einops.rearrange(loss_change_by_token, "batch token item -> (batch item) token")
loss_change_by_token_by_row = loss_change_by_token_by_row.unsqueeze(2)
loss_change_by_token_by_row.shape

torch.Size([100, 1024, 1])

In [120]:
save_array(loss_change_by_token_by_row, 'loss_change_by_token_by_row_hp.npy', model)

'data/pythia-2.8b/loss_change_by_token_by_row_hp.npy'

In [121]:
loss_change_by_token_by_row.sum()

tensor(71.0874)

In [47]:
from utils.neuroscope import plot_topk
loss_change_by_token = torch.from_numpy(load_array('loss_change_by_token_by_row_hp.npy', model))
plot_topk(
    activations=loss_change_by_token_by_row, 
    dataloader=subset_data_loader, 
    window_size=40, 
    model=model, 
    k=15, 
    centred=False, 
    exclusions=[" '", " ,", ",", "."," ."," Fig", "'t", " Pinterest", " Kampf", "m", "uk", " Kamp", "com", "edu", "S", "youtube", "twitter", "0", "js", "py", " Protein", " Fiber", " Carbohydrates", " Sugar", " Grant", " Pub", ","]
)

IndexError: too many indices for tensor of dimension 2

## Comma Ablation for Classification

In [58]:
import torch

import random

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
from transformer_lens import HookedTransformer
from datasets import load_dataset, Dataset, DatasetDict
from tqdm.notebook import tqdm
from utils.store import load_pickle, load_array
from utils.ablation import ablate_resid_with_precalc_mean

In [8]:
dataset = load_dataset("imdb")
dataset["train"][100]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

### GPT-2 Classifier

In [9]:
MODEL_NAME = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(8000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
model = AutoModelForCausalLM.from_pretrained("./gpt2_imdb_classifier")
class_layer_weights = load_pickle("gpt2_imdb_classifier_classification_head_weights", 'gpt2')

model = HookedTransformer.from_pretrained(
    "gpt2",
    hf_model=model,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [45]:
def get_classification_prediction(eval_dataset, dataset_idx, verbose=False):

    logits, cache = model.run_with_cache(small_eval_dataset[dataset_idx]['text'])
    last_token_act = cache['ln_final.hook_normalized'][0, -1, :]
    res = torch.softmax(torch.tensor(class_layer_weights['score.weight']) @ last_token_act.cpu(), dim=-1)
    if verbose:
        print(f"Sentence: {small_eval_dataset[dataset_idx]['text']}")
        print(f"Prediction: {res.argmax()} Label: {small_eval_dataset[dataset_idx]['label']}")

    return res.argmax(), small_eval_dataset[dataset_idx]['label'], res

In [47]:
def get_accuracy(eval_dataset, n=100):
    correct = 0
    for idx in range(min(len(eval_dataset), n)):
        pred, label, _ = get_classification_prediction(eval_dataset, idx)
        if pred == label:
            correct += 1
    return correct / n

get_accuracy(small_eval_dataset)

0.89

In [56]:
def compute_mean_ablation_modified_accuracy(model: HookedTransformer, dataset: Dataset, target_token_ids) -> float:

    for _, item in tqdm(enumerate(dataset), total=len(dataset)):
        batch_tokens = model.to_tokens(item['text'], prepend_bos=False)
        print(batch_tokens.shape)

    #     # get positions of all 13 and 15 token ids in batch
    #     punct_pos = find_positions(batch_tokens, token_ids=target_token_ids)

    #     # get the loss for each token in the batch
    #     initial_loss = model(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)
    #     orig_loss_list.append(initial_loss)
        
    #     # add hooks for the activations of the 13 and 15 tokens
    #     for layer, head in heads_to_ablate:
    #         mean_ablate_comma = partial(ablate_resid_with_precalc_mean, cached_means=cached_means, pos_by_batch=punct_pos, layer=layer)
    #         model.blocks[layer].hook_resid_post.add_hook(mean_ablate_comma)

    #     # get the loss for each token when run with hooks
    #     hooked_loss = model.run_with_cache(batch_tokens, return_type="loss", prepend_bos=False, loss_per_token=True)

    #     # compute the percent difference between the two losses
    #     loss_diff = hooked_loss - initial_loss
    #     loss_diff_list.append(loss_diff)

    # model.reset_hooks()
    # return loss_diff_list, orig_loss_list

In [54]:
# load the files
comma_mean_values = torch.from_numpy(load_array('comma_mean_values.npy', model)).to(device)
period_mean_values = torch.from_numpy(load_array('period_mean_values.npy', model)).to(device)

FileNotFoundError: [Errno 2] No such file or directory: 'data/gpt2-small/comma_mean_values.npy'

In [59]:
compute_mean_ablation_modified_accuracy(model, small_eval_dataset, target_token_ids=[15])

  0%|          | 0/2000 [00:00<?, ?it/s]

torch.Size([1, 413])
torch.Size([1, 230])
torch.Size([1, 157])
torch.Size([1, 139])
torch.Size([1, 108])
torch.Size([1, 171])
torch.Size([1, 147])
torch.Size([1, 164])
torch.Size([1, 177])
torch.Size([1, 158])
torch.Size([1, 173])
torch.Size([1, 154])
torch.Size([1, 274])
torch.Size([1, 340])
torch.Size([1, 606])
torch.Size([1, 232])
torch.Size([1, 217])
torch.Size([1, 344])
torch.Size([1, 193])
torch.Size([1, 229])
torch.Size([1, 175])
torch.Size([1, 581])
torch.Size([1, 634])
torch.Size([1, 206])
torch.Size([1, 161])
torch.Size([1, 296])
torch.Size([1, 225])
torch.Size([1, 163])
torch.Size([1, 245])
torch.Size([1, 166])
torch.Size([1, 438])
torch.Size([1, 153])
torch.Size([1, 114])
torch.Size([1, 94])
torch.Size([1, 209])
torch.Size([1, 75])
torch.Size([1, 156])
torch.Size([1, 385])
torch.Size([1, 209])
torch.Size([1, 666])
torch.Size([1, 67])
torch.Size([1, 137])
torch.Size([1, 423])
torch.Size([1, 151])
torch.Size([1, 142])
torch.Size([1, 208])
torch.Size([1, 270])
torch.Size([1, 2